In [1]:
#加载tokenizer
from transformers import AutoTokenizer
pretrained_model_name_or_path = r'C:\Users\peixi\Downloads\Huggingface\model\hflchinese-roberta-wwm-ext'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path)

import torch
from datasets import load_from_disk

class Dataset(torch.utils.data.Dataset):
    def __init__(self, split, max_samples=None):
        self.dataset = load_from_disk(r'C:\Users\peixi\Downloads\Huggingface\Data\ChnSentiCorp')[split]
        if max_samples:
            self.dataset = self.dataset.select(list(range(min(max_samples, len(self.dataset)))))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        review = self.dataset[i]['text']
        label = self.dataset[i]['label']
        return review, label

dataset = Dataset('train', 9600)
len(dataset), dataset[20]

#定义计算设备
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# 数据整理函数
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]
    # 编码
    data = tokenizer.batch_encode_plus(batch_text_or_text_pairs=sents, truncation=True, padding='max_length', max_length=500, return_tensors='pt', return_length=True)
    # input_ids：编码之后的数字
    # attention_mask：补零的位置是0, 其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)
    # 把数据移动到计算设备上
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)
    return input_ids, attention_mask, token_type_ids, labels



In [2]:
#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

len(loader)

600

In [3]:
#加载模型
from transformers import BertModel
pretrained_model_name_or_path = r'C:\Users\peixi\Downloads\Huggingface\model\hflchinese-roberta-wwm-ext'
pretrained = BertModel.from_pretrained(pretrained_model_name_or_path)

In [4]:
#设定计算设备
pretrained.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [5]:
# 预训练模型试算
for input_ids, attention_mask, token_type_ids, labels in loader:
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)

    out = pretrained(input_ids=input_ids,
                     attention_mask=attention_mask,
                     token_type_ids=token_type_ids)

    print(out.last_hidden_state.shape)
    break  # 退出循环

torch.Size([16, 500, 768])


In [6]:
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(in_features=768, out_features=2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        #使用预训练模型抽取数据特征
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        #对抽取的特征只取第一个字的结果做分类即可
        out = self.fc(out.last_hidden_state[:, 0])

        out = out.softmax(dim=1)

        return out


model = Model()

#设定计算设备
model.to(device)

#试算
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

torch.Size([16, 2])

In [7]:
#训练
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR

def train():
    # 定义优化器
    optimizer = optim.AdamW(model.parameters(), lr=5e-4)

    # 定义loss函数
    criterion = torch.nn.CrossEntropyLoss()

    # 定义学习率调节器
    scheduler = LambdaLR(optimizer, lr_lambda=lambda step: 0.98 ** step)  # 使用LambdaLR作为学习率调节器

    # 模型切换到训练模式
    model.train()

    # 按批次遍历训练集中的数据
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):

        # 模型计算
        out = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # 计算loss并使用梯度下降法优化模型参数
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # 输出各项数据的情况，便于观察
        if i % 10 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            lr = optimizer.param_groups[0]['lr']
            print(i, loss.item(), lr, accuracy)

train()

0 0.6771042943000793 0.00049 0.4375
10 0.4872928559780121 0.00040036567537489795 0.8125
20 0.4319842755794525 0.00032712790615999613 0.875
30 0.4958016574382782 0.00026728731649739405 0.75
40 0.4494304656982422 0.00021839319793596584 0.875
50 0.3881189227104187 0.00017844314324268721 0.9375
60 0.46474751830101013 0.00014580103991913902 0.8125
70 0.46824130415916443 0.00011913006493385416 0.875
80 0.41674932837486267 9.733793654019992e-05 0.875
90 0.47692564129829407 7.953218102554305e-05 0.75
100 0.3346036970615387 6.498358238842894e-05 1.0
110 0.4870145916938782 5.309631806372179e-05 0.875
120 0.4098713994026184 4.3383557635719116e-05 0.875
130 0.44377946853637695 3.544752521018468e-05 0.8125
140 0.4476511478424072 2.8963208920702668e-05 0.875
150 0.4154006838798523 2.366505040930896e-05 0.9375
160 0.32949233055114746 1.933606916306936e-05 1.0
170 0.5102009177207947 1.5798976305240818e-05 0.8125
180 0.5902838110923767 1.2908913915672958e-05 0.6875
190 0.44055911898612976 1.05475225269

In [8]:
#测试
def test():
    #定义测试数据集加载器
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=16,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    #下游任务模型切换到运行模式
    model.eval()
    correct = 0
    total = 0
    sentence_count = 0  # 用于记录句子数量

    #增加输出前5句的结果并与真实的label进行比较
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):

        #计算
        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        #统计正确率
        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

        #输出前5句的结果并与真实的label进行比较
        for j in range(len(input_ids)):  # 遍历当前batch的每个句子
            # Decode input_ids
            decoded_input_ids = tokenizer.decode(input_ids[j], skip_special_tokens=True)
            print("Review:", decoded_input_ids)  # 输出 input_ids 的 decode 结果
            print("Prediction:", out[j].item())  # 输出预测结果
            print("True label:", labels[j].item())  # 输出真实标签
            sentence_count += 1  # 句子数量加1
            if sentence_count == 5:  # 当输出了5条句子后，退出循环
                break

        if sentence_count == 5:  # 当输出了5条句子后，退出外层循环
            break

    print("Accuracy for the first 5 sentences:", correct / total)

test()

Review: 性 价 比 极 高 ， 我 在 苏 宁 买 4699 ， 东 东 才 4399. 功 能 很 全 ， 用 起 来 很 满 意 ， 够 用 了 。
Prediction: 1
True label: 1
Review: 看 完 之 后 ， 非 常 喜 欢 作 者 细 腻 的 情 感 和 文 笔 ， 她 把 都 市 生 活 中 可 能 遇 到 的 很 多 情 况 在 小 说 主 人 公 身 上 再 现 了 ， 让 读 书 的 我 们 感 同 身 受 ， 并 且 针 对 这 些 问 题 ， 针 对 我 们 经 常 会 出 现 的 负 面 情 绪 找 到 内 心 深 处 最 根 本 的 原 因 ， 印 证 了 作 者 的 观 点 ： 当 外 境 有 任 何 东 西 触 动 你 的 时 候 ， 记 得 ， 要 忘 往 内 看 。 全 书 看 完 ， 我 的 心 情 非 常 平 静 ， 那 种 感 觉 我 不 知 道 要 怎 样 表 述 ， 好 像 有 种 淡 淡 的 甜 味 ， 很 淡 ， 不 腻 人 ， 很 美 好 的 感 觉 。
Prediction: 1
True label: 1
Review: 说 实 话 ， 写 的 实 在 不 怎 么 样 ， 东 拼 西 凑 ， 主 人 公 那 样 的 心 理 素 质 还 当 心 理 师 ， 实 在 米 名 奇 妙
Prediction: 0
True label: 0
Review: 优 点 不 用 说 了 ， 谁 让 咱 是 穷 人 价 格 是 很 吸 引 人 的 因 素 。 我 只 说 说 缺 点 ， 拿 到 手 后 发 现 风 扇 发 出 很 尖 锐 的 高 频 声 音 ， 安 静 的 时 候 很 是 不 爽 。 一 气 之 下 我 把 它 给 拆 了 ， 应 该 是 装 配 问 题 ， 风 扇 运 转 很 安 静 ， 是 其 他 地 方 和 它 发 生 了 共 振 或 摩 擦 ， 从 新 装 了 一 遍 现 在 好 了 。 底 部 外 壳 在 按 的 的 时 候 会 发 生 一 个 咔 的 声 音 ， 经 过 仔 细 观 察 ， 是 底 壳 和 板 子 边 缘 产 生 的 ， 稍 微 处 理 一 下 接 触 部 位 解 决 。
Prediction: 0
True label: 1
Review: 非 常

In [9]:
#手动保存模型
pretrained.save_pretrained(r'C:\Users\peixi\Downloads\Huggingface\model\hflchinese-roberta-wwm-ext\trained')